In [14]:
#分类训练模型  1. service_type=3 <=> current_service=99104722
#分类训练模型  2. service_type=1 <=> current_service={90063345: 0, 90109916: 1, 90155946: 2}
#分类训练模型  3. service_type=4 <=> current_service={89016252: 0, 89016253: 1, 89016259: 2, 89950166: 3, 89950167: 4, 89950168: 5, 99999825: 6, 99999826: 7, 99999827: 8, 99999828: 9, 99999830: 10}


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import warnings
from sklearn.preprocessing import OneHotEncoder
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
n_splits = 5
seed = 35
path = 'E:/BDCI Match/'

#文件路径
traindataFile = path + 'train.csv'
testdataFile = path+ 'test.csv'

#文件加载
train = pd.read_csv(traindataFile)
test = pd.read_csv(testdataFile)

#current_service为99104722的记录可以剔除，因为service_type=3 <=> current_service=99104722
train1 = train.loc[train['service_type']==1]
train1 = train1.reset_index()

train2 = train.loc[train['service_type']==4]
train2 = train2.reset_index()

test1 = test.loc[test['service_type']==1]
test1 = test1.reset_index()
test2 = test.loc[test['service_type']==4]
test2 = test2.reset_index()
test3 = test.loc[test['service_type']==3]
test3 = test3.reset_index()
test_id3 = test3['user_id']

# 对标签编码 映射关系
label2current_service1 = dict(zip(range(0,len(set(train1['current_service']))),sorted(list(set(train1['current_service'])))))
current_service2label1 = dict(zip(sorted(list(set(train1['current_service']))),range(0,len(set(train1['current_service'])))))

label2current_service2 = dict(zip(range(0,len(set(train2['current_service']))),sorted(list(set(train2['current_service'])))))
current_service2label2 = dict(zip(sorted(list(set(train2['current_service']))),range(0,len(set(train2['current_service'])))))

# 原始数据的标签映射
train1['current_service'] = train1['current_service'].map(current_service2label1)
train2['current_service'] = train2['current_service'].map(current_service2label2)

print('train1 data shape',train1.shape)
print('train1 data of user_id shape',len(set(train1['user_id'])))
print('train1 data of current_service shape',current_service2label1)

print('test1 data shape',test1.shape)
print('test1 data of user_id shape',len(set(test1['user_id'])))

print('train2 data shape',train2.shape)
print('train2 data of user_id shape',len(set(train2['user_id'])))
print('train2 data of current_service shape',current_service2label2)

print('test2 data shape',test2.shape)
print('test2 data of user_id shape',len(set(test2['user_id'])))

print('test3 data shape',test3.shape)
print('test3 data of user_id shape',len(set(test3['user_id'])))


# t1 = train1.loc[(train1['current_service']!=0)&(train1['current_service']!=1)&(train1['current_service']!=2)]
# print(t1)
# exit()

# 构造原始数据
index1 = train1.pop('index')
y1 = train1.pop('current_service')
train_id1 = train1.pop('user_id')
train1.pop('service_type')
X1 = train1
train_col = train1.columns

X1_test = test1[train_col]
test_id1 = test1['user_id']

# 构造原始数据
index2 = train2.pop('index')
y2 = train2.pop('current_service')
train_id2 = train2.pop('user_id')
train2.pop('service_type')
X2 = train2
train_col = train2.columns

X2_test = test2[train_col]
test_id2 = test2['user_id']

# 数据有问题数据
for i in train_col:
    X1[i] = X1[i].replace("\\N",-1)
    X2[i] = X2[i].replace("\\N",-1)
    X1_test[i] = X1_test[i].replace("\\N",-1)
    X2_test[i] = X2_test[i].replace("\\N",-1)

    
X1,y1,X1_test = X1.values,y1,X1_test.values
X2,y2,X2_test = X2.values,y2,X2_test.values


# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

# lgb 参数
params1={
    "learning_rate":0.02,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":-1,
    "objective":"multiclass",
    "num_class":3,
    "silent":True,
}

params2={
    "learning_rate":0.02,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":-1,
    "objective":"multiclass",
    "num_class":11,
    "silent":True,
}

# 自定义F1评价函数
def f1_score_vali3(preds, data_vali):
#     print(preds.shape)
#     print(preds)
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

# 自定义F1评价函数
def f1_score_vali11(preds, data_vali):
#     print(preds.shape)
#     print(preds)
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(11, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

#k折训练模型
def trainKfold(X, y, X_test, params):
    xx_score = []
    cv_pred = []
   
    skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
    for index,(train_index,test_index) in enumerate(skf.split(X,y)):
        print(index)

        X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        validation_data = lgb.Dataset(X_valid, label=y_valid)
        if params['num_class'] ==3:
            clf=lgb.train(params,train_data,num_boost_round=10000,valid_sets=[validation_data],early_stopping_rounds=100,feval=f1_score_vali3,verbose_eval=20)
        else:
            clf=lgb.train(params,train_data,num_boost_round=10000,valid_sets=[validation_data],early_stopping_rounds=100,feval=f1_score_vali11,verbose_eval=20)

        xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

        xx_pred = [np.argmax(x) for x in xx_pred]

        xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

        y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

        y_test = [np.argmax(x) for x in y_test]

        if index == 0:
            cv_pred = np.array(y_test).reshape(-1, 1)
        else:
            cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

    # 投票
    submit = []
    for line in cv_pred:
        submit.append(np.argmax(np.bincount(line)))
    return submit

submit1 = trainKfold(X1, y1, X1_test, params1)
submit2 = trainKfold(X2, y2, X2_test, params2)
submit3 = [99104722]*len(list(test_id3.unique()))


temp1 = pd.DataFrame()
temp2 = pd.DataFrame()
temp1['submit1'] = submit1
temp1['submit1'] = temp1['submit1'].map(label2current_service1)
temp2['submit2'] = submit2
temp2['submit2'] = temp2['submit2'].map(label2current_service2)

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id1.unique())+list(test_id2.unique())+list(test_id3.unique())
df_test['predict'] = list(temp1['submit1'])+list(temp2['submit2'])+list(submit3)
# df_test['predict'] = df_test['predict'].map(label2current_service)

df_test.to_csv(path+ './result/sample_submission.csv',index=False)


train1 data shape (243407, 28)
train1 data of user_id shape 243407
train1 data of current_service shape {90063345: 0, 90109916: 1, 90155946: 2}
test1 data shape (103945, 27)
test1 data of user_id shape 103945
train2 data shape (332956, 28)
train2 data of user_id shape 332956
train2 data of current_service shape {89016252: 0, 89016253: 1, 89016259: 2, 89950166: 3, 89950167: 4, 89950168: 5, 99999825: 6, 99999826: 7, 99999827: 8, 99999828: 9, 99999830: 10}
test2 data shape (142914, 27)
test2 data of user_id shape 142914
test3 data shape (15706, 27)
test3 data of user_id shape 15706
0
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's multi_logloss: 0.674453	valid_0's f1_score: 0.987413
[40]	valid_0's multi_logloss: 0.437274	valid_0's f1_score: 0.988096
[60]	valid_0's multi_logloss: 0.293281	valid_0's f1_score: 0.989051
[80]	valid_0's multi_logloss: 0.202029	valid_0's f1_score: 0.990201
[100]	valid_0's multi_logloss: 0.142735	valid_0's f1_score: 0.990612
[120]	va

[1160]	valid_0's multi_logloss: 0.0122333	valid_0's f1_score: 0.995501
[1180]	valid_0's multi_logloss: 0.0122248	valid_0's f1_score: 0.995521
[1200]	valid_0's multi_logloss: 0.0122117	valid_0's f1_score: 0.995542
[1220]	valid_0's multi_logloss: 0.0121992	valid_0's f1_score: 0.995522
[1240]	valid_0's multi_logloss: 0.0121863	valid_0's f1_score: 0.995502
[1260]	valid_0's multi_logloss: 0.0121772	valid_0's f1_score: 0.995481
Early stopping, best iteration is:
[1177]	valid_0's multi_logloss: 0.0122269	valid_0's f1_score: 0.995563
2
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's multi_logloss: 0.674521	valid_0's f1_score: 0.986633
[40]	valid_0's multi_logloss: 0.437075	valid_0's f1_score: 0.988362
[60]	valid_0's multi_logloss: 0.292889	valid_0's f1_score: 0.989734
[80]	valid_0's multi_logloss: 0.20151	valid_0's f1_score: 0.990589
[100]	valid_0's multi_logloss: 0.142096	valid_0's f1_score: 0.990924
[120]	valid_0's multi_logloss: 0.10261	valid_0's f1_score: 0.99

[240]	valid_0's multi_logloss: 0.0260022	valid_0's f1_score: 0.993385
[260]	valid_0's multi_logloss: 0.0229101	valid_0's f1_score: 0.993487
[280]	valid_0's multi_logloss: 0.0206889	valid_0's f1_score: 0.993716
[300]	valid_0's multi_logloss: 0.0190318	valid_0's f1_score: 0.993942
[320]	valid_0's multi_logloss: 0.0178139	valid_0's f1_score: 0.994127
[340]	valid_0's multi_logloss: 0.0168711	valid_0's f1_score: 0.994252
[360]	valid_0's multi_logloss: 0.0161455	valid_0's f1_score: 0.994417
[380]	valid_0's multi_logloss: 0.0155695	valid_0's f1_score: 0.994559
[400]	valid_0's multi_logloss: 0.015118	valid_0's f1_score: 0.994559
[420]	valid_0's multi_logloss: 0.0147127	valid_0's f1_score: 0.994723
[440]	valid_0's multi_logloss: 0.0143977	valid_0's f1_score: 0.994846
[460]	valid_0's multi_logloss: 0.0141455	valid_0's f1_score: 0.994928
[480]	valid_0's multi_logloss: 0.0139447	valid_0's f1_score: 0.99501
[500]	valid_0's multi_logloss: 0.0137811	valid_0's f1_score: 0.995031
[520]	valid_0's multi_

[1420]	valid_0's multi_logloss: 0.349901	valid_0's f1_score: 0.871176
[1440]	valid_0's multi_logloss: 0.349417	valid_0's f1_score: 0.871265
[1460]	valid_0's multi_logloss: 0.348894	valid_0's f1_score: 0.871422
[1480]	valid_0's multi_logloss: 0.348414	valid_0's f1_score: 0.87159
[1500]	valid_0's multi_logloss: 0.347944	valid_0's f1_score: 0.871632
[1520]	valid_0's multi_logloss: 0.347483	valid_0's f1_score: 0.871863
[1540]	valid_0's multi_logloss: 0.347028	valid_0's f1_score: 0.871964
[1560]	valid_0's multi_logloss: 0.346568	valid_0's f1_score: 0.871972
[1580]	valid_0's multi_logloss: 0.34616	valid_0's f1_score: 0.872058
[1600]	valid_0's multi_logloss: 0.345716	valid_0's f1_score: 0.872489
[1620]	valid_0's multi_logloss: 0.345299	valid_0's f1_score: 0.872739
[1640]	valid_0's multi_logloss: 0.344877	valid_0's f1_score: 0.872941
[1660]	valid_0's multi_logloss: 0.344498	valid_0's f1_score: 0.873218
[1680]	valid_0's multi_logloss: 0.344081	valid_0's f1_score: 0.873494
[1700]	valid_0's multi

[3780]	valid_0's multi_logloss: 0.319991	valid_0's f1_score: 0.882646
[3800]	valid_0's multi_logloss: 0.319847	valid_0's f1_score: 0.882772
[3820]	valid_0's multi_logloss: 0.319732	valid_0's f1_score: 0.882921
[3840]	valid_0's multi_logloss: 0.319623	valid_0's f1_score: 0.882842
[3860]	valid_0's multi_logloss: 0.319511	valid_0's f1_score: 0.882955
[3880]	valid_0's multi_logloss: 0.31937	valid_0's f1_score: 0.883077
[3900]	valid_0's multi_logloss: 0.319257	valid_0's f1_score: 0.883118
[3920]	valid_0's multi_logloss: 0.319122	valid_0's f1_score: 0.883285
[3940]	valid_0's multi_logloss: 0.319	valid_0's f1_score: 0.883197
[3960]	valid_0's multi_logloss: 0.318893	valid_0's f1_score: 0.883309
[3980]	valid_0's multi_logloss: 0.318767	valid_0's f1_score: 0.88345
[4000]	valid_0's multi_logloss: 0.318656	valid_0's f1_score: 0.88352
[4020]	valid_0's multi_logloss: 0.318546	valid_0's f1_score: 0.883477
[4040]	valid_0's multi_logloss: 0.31843	valid_0's f1_score: 0.883398
[4060]	valid_0's multi_logl

...

[1700]	valid_0's multi_logloss: 0.342147	valid_0's f1_score: 0.874668
[1720]	valid_0's multi_logloss: 0.341798	valid_0's f1_score: 0.874818
[1740]	valid_0's multi_logloss: 0.341448	valid_0's f1_score: 0.875132
[1760]	valid_0's multi_logloss: 0.341092	valid_0's f1_score: 0.875181
[1780]	valid_0's multi_logloss: 0.340748	valid_0's f1_score: 0.875409
[1800]	valid_0's multi_logloss: 0.340374	valid_0's f1_score: 0.875449
[1820]	valid_0's multi_logloss: 0.34001	valid_0's f1_score: 0.875763
[1840]	valid_0's multi_logloss: 0.339629	valid_0's f1_score: 0.875962
[1860]	valid_0's multi_logloss: 0.339339	valid_0's f1_score: 0.87595
[1880]	valid_0's multi_logloss: 0.339007	valid_0's f1_score: 0.87625
[1900]	valid_0's multi_logloss: 0.338698	valid_0's f1_score: 0.876331
[1920]	valid_0's multi_logloss: 0.33835	valid_0's f1_score: 0.876587
[1940]	valid_0's multi_logloss: 0.338	valid_0's f1_score: 0.876452
[1960]	valid_0's multi_logloss: 0.337654	valid_0's f1_score: 0.876549
[1980]	valid_0's multi_logl

[4060]	valid_0's multi_logloss: 0.317845	valid_0's f1_score: 0.885012
[4080]	valid_0's multi_logloss: 0.317709	valid_0's f1_score: 0.884954
[4100]	valid_0's multi_logloss: 0.317584	valid_0's f1_score: 0.885005
[4120]	valid_0's multi_logloss: 0.317462	valid_0's f1_score: 0.885142
[4140]	valid_0's multi_logloss: 0.317344	valid_0's f1_score: 0.885184
[4160]	valid_0's multi_logloss: 0.317233	valid_0's f1_score: 0.885188
[4180]	valid_0's multi_logloss: 0.317132	valid_0's f1_score: 0.885323
[4200]	valid_0's multi_logloss: 0.317012	valid_0's f1_score: 0.885219
[4220]	valid_0's multi_logloss: 0.3169	valid_0's f1_score: 0.885273
[4240]	valid_0's multi_logloss: 0.316789	valid_0's f1_score: 0.885286
[4260]	valid_0's multi_logloss: 0.316686	valid_0's f1_score: 0.885358
[4280]	valid_0's multi_logloss: 0.316594	valid_0's f1_score: 0.88544
[4300]	valid_0's multi_logloss: 0.316484	valid_0's f1_score: 0.885452
[4320]	valid_0's multi_logloss: 0.3164	valid_0's f1_score: 0.885351
[4340]	valid_0's multi_lo

[40]	valid_0's multi_logloss: 1.24715	valid_0's f1_score: 0.800888
[60]	valid_0's multi_logloss: 1.02348	valid_0's f1_score: 0.807984
[80]	valid_0's multi_logloss: 0.872245	valid_0's f1_score: 0.814589
[100]	valid_0's multi_logloss: 0.766247	valid_0's f1_score: 0.819044
[120]	valid_0's multi_logloss: 0.689161	valid_0's f1_score: 0.823635
[140]	valid_0's multi_logloss: 0.632143	valid_0's f1_score: 0.82825
[160]	valid_0's multi_logloss: 0.58967	valid_0's f1_score: 0.831081
[180]	valid_0's multi_logloss: 0.556532	valid_0's f1_score: 0.83453
[200]	valid_0's multi_logloss: 0.530499	valid_0's f1_score: 0.836826
[220]	valid_0's multi_logloss: 0.509081	valid_0's f1_score: 0.839459
[240]	valid_0's multi_logloss: 0.49216	valid_0's f1_score: 0.841582
[260]	valid_0's multi_logloss: 0.478119	valid_0's f1_score: 0.843384
[280]	valid_0's multi_logloss: 0.466531	valid_0's f1_score: 0.84532
[300]	valid_0's multi_logloss: 0.456836	valid_0's f1_score: 0.846634
[320]	valid_0's multi_logloss: 0.448301	vali

[2420]	valid_0's multi_logloss: 0.33324	valid_0's f1_score: 0.880576
[2440]	valid_0's multi_logloss: 0.332981	valid_0's f1_score: 0.880636
[2460]	valid_0's multi_logloss: 0.33272	valid_0's f1_score: 0.880597
[2480]	valid_0's multi_logloss: 0.332482	valid_0's f1_score: 0.88063
[2500]	valid_0's multi_logloss: 0.332233	valid_0's f1_score: 0.88087
[2520]	valid_0's multi_logloss: 0.33198	valid_0's f1_score: 0.880842
[2540]	valid_0's multi_logloss: 0.331746	valid_0's f1_score: 0.880819
[2560]	valid_0's multi_logloss: 0.331478	valid_0's f1_score: 0.880956
[2580]	valid_0's multi_logloss: 0.331268	valid_0's f1_score: 0.88111
[2600]	valid_0's multi_logloss: 0.33104	valid_0's f1_score: 0.881165
[2620]	valid_0's multi_logloss: 0.3308	valid_0's f1_score: 0.881242
[2640]	valid_0's multi_logloss: 0.330572	valid_0's f1_score: 0.881349
[2660]	valid_0's multi_logloss: 0.33036	valid_0's f1_score: 0.881323
[2680]	valid_0's multi_logloss: 0.330133	valid_0's f1_score: 0.8812
[2700]	valid_0's multi_logloss: 

[4780]	valid_0's multi_logloss: 0.315443	valid_0's f1_score: 0.886821
[4800]	valid_0's multi_logloss: 0.315362	valid_0's f1_score: 0.886812
[4820]	valid_0's multi_logloss: 0.315284	valid_0's f1_score: 0.886933
[4840]	valid_0's multi_logloss: 0.315203	valid_0's f1_score: 0.886964
[4860]	valid_0's multi_logloss: 0.315118	valid_0's f1_score: 0.887108
[4880]	valid_0's multi_logloss: 0.315049	valid_0's f1_score: 0.887112
[4900]	valid_0's multi_logloss: 0.314967	valid_0's f1_score: 0.887146
[4920]	valid_0's multi_logloss: 0.314895	valid_0's f1_score: 0.887164
[4940]	valid_0's multi_logloss: 0.31481	valid_0's f1_score: 0.887189
[4960]	valid_0's multi_logloss: 0.314728	valid_0's f1_score: 0.887296
[4980]	valid_0's multi_logloss: 0.314649	valid_0's f1_score: 0.887357
[5000]	valid_0's multi_logloss: 0.314573	valid_0's f1_score: 0.887468
[5020]	valid_0's multi_logloss: 0.314509	valid_0's f1_score: 0.887374
[5040]	valid_0's multi_logloss: 0.314446	valid_0's f1_score: 0.887422
[5060]	valid_0's mult